# Excercise 3 Image Filtering

In [ ]:
import scipy.ndimage as ndi
import numpy as np
import matplotlib.pyplot as plt
import skimage.io as io

### Smoothing filters

In [ ]:
# Read image from disk and convert grey levels to double
x = np.float64(io.imread("images/lena.jpg"))

# Create the filter kernel
k = 3
h = np.ones((k, k)) / (k ** 2) # mean filter kernel

# Compute the filter output
y = ndi.correlate(x, h, mode="reflect")
plt.figure(1)
plt.subplot(1, 2, 1)
plt.title("Original image")
plt.imshow(x, clim=[0, 255], cmap="gray")
plt.subplot(1, 2, 2)
plt.title(f"Mean filtered image k = {k}")
plt.imshow(y, clim=[0, 255], cmap="gray")

### Gaussian filter

In [ ]:
# Read image from disk and convert grey levels to double
x = np.float64(io.imread("images/lena.jpg"))
y = ndi.gaussian_filter(x, sigma=5, order=0) # TODO: check on documentation
plt.figure(1)
plt.subplot(1, 2, 1)
plt.title("Original image")
plt.imshow(x, clim=[0, 255], cmap="gray")
plt.subplot(1, 2, 2)
plt.title(f"Gaussian filtered image")
plt.imshow(y, clim=[0, 255], cmap="gray")

### Smoothing and tresholding
An important application of spatial averaging filters is to blur the image so that small, uninteresting objects are confused with the background and to emphasize larger objects, which can then be easily detected.

In [ ]:
x = np.float64(io.imread("images/space.jpg"))

# Compute a mean filter with a kernel size of 15x15
k = 15
h = np.ones((k, k)) / (k ** 2)
y = ndi.correlate(x, h, mode="reflect")

# Compute the maximum grey level
max_grey = np.max(y)

# Compute a mask that eliminates all grey levels below 25% of max value
mask = y > (max_grey * (25.0 / 100.0))

# Compute the tresholding
z = mask * x

# Show results
plt.figure(1)
plt.subplot(1, 3, 1)
plt.title("Original image")
plt.imshow(x, clim=[0, 255], cmap="gray")
plt.subplot(1, 3, 2)
plt.title(f"Mean filtered image")
plt.imshow(y, clim=[0, 255], cmap="gray")
plt.subplot(1, 3, 3)
plt.title(f"Tresholded image at 25%")
plt.imshow(z, clim=[0, 255], cmap="gray")
plt.subplots_adjust(wspace=0.8, hspace=0.8)

### Denoising

In [ ]:
x = np.float64(io.imread("images/lena.jpg"))
d = 25.0 # noise std
M, N = x.shape
noisy = x + d * np.random.randn(M, N)

plt.figure()
plt.title("Original image")
plt.imshow(x, clim=[0, 255], cmap="gray")
plt.figure()
plt.title(f"Noised image (std = {d})")
plt.imshow(noisy, cmap="gray")

# Denoising
k = 5
y = ndi.uniform_filter(noisy, k)
mse = np.mean(((x - y) ** 2)) # Compute the MSE between x and y
plt.figure()
plt.title(f"Denoised image (uniform filter, k = {k}, mse = {mse})")
plt.imshow(y, clim=[0, 255], cmap="gray")


### Spatial adaptive filter

In [ ]:
x = np.float64(io.imread("images/barbara.png"))
M, N = x.shape
curve = []

for d in range(5, 45, 5):
	k = 7
	var = d ** 2
	noisy = x + d * np.random.randn(M, N)
	local_mean = ndi.generic_filter(noisy, np.mean, (k,k))
	local_var = ndi.generic_filter(noisy, np.var, (k,k))
	out = noisy - (var / local_var) * (noisy - local_mean)
	mse = np.mean((x - out) ** 2)
	curve.append((d, mse))
	plt.figure()
	plt.title(f"Filtered image with d = {d} and k = {k}")
	plt.imshow(out, cmap="gray", clim=[0, 255])

a,b = zip(*curve)
plt.figure()
plt.stem(a, b)
plt.xlabel('Noise Sigma')
plt.ylabel('MSE')
plt.title('MSE vs Sigma Noise')
plt.grid(True)
plt.show()

### Point and line detection

In [ ]:
# Point detection
x = np.float64(io.imread("images/turbina.jpg"))
h_line = np.array([
	[-1, -1, -1],
	[-1, 8, -1],
	[-1, -1, -1]
])
y = ndi.correlate(x, h_line)
y = abs(y)
mask = y > (0.9 * np.max(y))
y = mask * y

plt.figure()
plt.imshow(x, clim=[0, 255], cmap="gray")
plt.figure()
plt.imshow(y, cmap="gray")

In [ ]:
# Line detection
x = np.float64(io.imread("images/quadrato.jpg"))
h_line_1 = np.array([
	[-1, -1, -1],
	[2, 2, 2],
	[-1, -1, -1]
])
h_line_2 = np.array([
	[-1, -1, 2],
	[-1, 2, -1],
	[2, -1, -1]
])
h_line_3 = np.array([
	[-1, 2, -1],
	[-1, 2, -1],
	[-1, 2, -1]
])
h_line_4 = np.array([
	[2, -1, -1],
	[-1, 2, -1],
	[-1, -1, 2]
])

y1 = np.abs(ndi.correlate(x, h_line_1))
y2 = np.abs(ndi.correlate(x, h_line_2))
y3 = np.abs(ndi.correlate(x, h_line_3))
y4 = np.abs(ndi.correlate(x, h_line_4))

y_stack = np.stack([y1, y2, y3, y4], axis=-1)
z = np.max(y_stack, 2)

plt.figure()
plt.imshow(x, clim=[0, 255], cmap="gray")
plt.figure()
plt.imshow(y1, cmap="gray")
plt.figure()
plt.imshow(y2, cmap="gray")
plt.figure()
plt.imshow(y3, cmap="gray")
plt.figure()
plt.imshow(y4, cmap="gray")
plt.figure()
plt.imshow(z, cmap="gray")
